<a href="https://colab.research.google.com/github/GitAmitesh/SkinDiseaseClassificationCNN/blob/main/model_1_sequential_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import keras

from scipy import stats
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical, Sequence
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import EarlyStopping

In [2]:
!fusermount -u drive
drive.mount('/content/drive')

fusermount: failed to unmount /content/drive: No such file or directory
Mounted at /content/drive


In [3]:
# Path to your .pkl file
file_path = "/content/drive/MyDrive/HAM10000/final_processed_ham_metadata.pkl"

# Load the DataFrame
df = pd.read_pickle(file_path)

# Check the result
df.head()


,lesion_id,image_id,dx,dx_type,age,sex,localization,image_path,lesion_type,lesion_type_idx,image_array
0,HAM_0006018,ISIC_0033486,bkl,histo,75.0,male,trunk,/content/drive/MyDrive/HAM10000/HAM10000_image...,Benign keratosis-like lesions,2,"[[[0.62352943, 0.58431375, 0.61960787], [0.619..."
1,HAM_0000206,ISIC_0033925,mel,histo,65.0,female,upper extremity,/content/drive/MyDrive/HAM10000/HAM10000_image...,Melanoma,5,"[[[0.7529412, 0.57254905, 0.654902], [0.745098..."
2,HAM_0000673,ISIC_0029659,akiec,histo,70.0,female,face,/content/drive/MyDrive/HAM10000/HAM10000_image...,Actinic keratoses,0,"[[[0.75686276, 0.4862745, 0.4862745], [0.73333..."
3,HAM_0000000,aug_normal_skin_0298,ns,none,45.0,unknown,unknown,/content/drive/MyDrive/HAM10000/normal_skin_im...,Normal skin,7,"[[[0.30980393, 0.23529412, 0.1764706], [0.3058..."
4,HAM_0005505,ISIC_0031743,akiec,histo,50.0,male,upper extremity,/content/drive/MyDrive/HAM10000/HAM10000_image...,Actinic keratoses,0,"[[[0.015686275, 0.011764706, 0.003921569], [0...."


In [6]:
# Constants
SIZE = (150, 200, 3)
num_classes = 8
image_shape = (SIZE[0], SIZE[1], SIZE[2])
image_array_path = "/content/drive/MyDrive/HAM10000/final_processed_image_arrays.npy"  # This should be pre-saved memmap file path

# Load memory-mapped image array (not in RAM)
num_images = len(df)
image_arrays_loaded = np.memmap(image_array_path, dtype=np.float32, mode='r', shape=(num_images, *image_shape))

# Labels
Y = df['lesion_type_idx']
Y_cat = to_categorical(Y, num_classes=num_classes)

# Train-test split (indices only)
train_idx, test_idx = train_test_split(np.arange(num_images), test_size=0.25, random_state=42)

# -----------------------------
# Custom Data Generator
# -----------------------------
class MemmapImageGenerator(Sequence):
    def __init__(self, memmap, labels, indices, batch_size=32):
        self.memmap = memmap
        self.labels = labels
        self.indices = indices
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.indices) / self.batch_size))

    def __getitem__(self, idx):
        batch_ids = self.indices[idx * self.batch_size : (idx + 1) * self.batch_size]
        batch_x = self.memmap[batch_ids]
        batch_y = self.labels[batch_ids]
        return batch_x, batch_y

# Create training and validation generators
batch_size = 64
train_gen = MemmapImageGenerator(image_arrays_loaded, Y_cat, train_idx, batch_size=batch_size)
test_gen = MemmapImageGenerator(image_arrays_loaded, Y_cat, test_idx, batch_size=batch_size)


model = Sequential()
model.add(Conv2D(256, (3, 3), activation="relu", input_shape=SIZE))
#model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3, 3),activation='relu'))
#model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(64, (3, 3),activation='relu'))
#model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Flatten())

model.add(Dense(32))
model.add(Dense(8, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['acc'])


# Train
#You can also use generator to use augmentation during training.

batch_size = 64
epochs = 50

early_stop = EarlyStopping(
    monitor='val_loss',        # What to monitor (e.g., val_loss or val_accuracy)
    patience=5,                # How many epochs to wait before stopping (if no improvement)
    restore_best_weights=True # Optional: roll back to best model after stopping
)

history = model.fit(
    train_gen,
    epochs = epochs,
    batch_size = batch_size,
    validation_data = test_gen,
    callbacks = [early_stop],
    verbose=2)

score = model.evaluate(test_gen)
print('Test accuracy:', score[1])



/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 198, 256)  │         7,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 99, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 74, 99, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 97, 128)    │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 48, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 36, 48, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 46, 64)     │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 23, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 17, 23, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25024)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │       800,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           264 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,177,064 (4.49 MB)

 Trainable params: 1,177,064 (4.49 MB)

 Non-trainable params: 0 (0.00 B)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
141/141 - 128s - 909ms/step - acc: 0.1648 - loss: 2.1677 - val_acc: 0.1608 - val_loss: 4353248575106597844418560.0000
Epoch 2/50
141/141 - 60s - 425ms/step - acc: 0.1683 - loss: 2.0722 - val_acc: 0.1608 - val_loss: 6276223358232550866681856.0000
Epoch 3/50
141/141 - 82s - 583ms/step - acc: 0.1684 - loss: 2.0718 - val_acc: 0.1608 - val_loss: 5856437144415417938214912.0000
Epoch 4/50
141/141 - 62s - 437ms/step - acc: 0.1684 - loss: 2.0718 - val_acc: 0.1608 - val_loss: 5839185979941985686913024.0000
Epoch 5/50
141/141 - 63s - 447ms/step - acc: 0.1684 - loss: 2.0718 - val_acc: 0.1608 - val_loss: 5785527860196078421803008.0000
Epoch 6/50
141/141 - 63s - 449ms/step - acc: 0.1684 - loss: 2.0719 - val_acc: 0.1608 - val_loss: 5911408441755072402030592.0000
47/47 ━━━━━━━━━━━━━━━━━━━━ 12s 254ms/step - acc: 0.1536 - loss: 3666392706433307241349120.0000
Test accuracy: 0.16083915531635284


In [ ]:

#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


acc = history.history['acc']
val_acc = history.history['val_acc']
plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


# Prediction on test data
y_pred = model.predict(x_test)
# Convert predictions classes to one hot vectors
y_pred_classes = np.argmax(y_pred, axis = 1)
# Convert test data to one hot vectors
y_true = np.argmax(y_test, axis = 1)

#Print confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)

fig, ax = plt.subplots(figsize=(6,6))
sns.set(font_scale=1.6)
sns.heatmap(cm, annot=True, linewidths=.5, ax=ax)


#PLot fractional incorrect misclassifications
incorr_fraction = 1 - np.diag(cm) / np.sum(cm, axis=1)
plt.bar(np.arange(7), incorr_fraction)
plt.xlabel('True Label')
plt.ylabel('Fraction of incorrect predictions')
